In [3]:
## Test Bicubic

In [4]:
from calulate_psnr_ssim import *

In [6]:
import cv2 as cv2
import numpy as np
import torch.nn.functional as F
import os

test_psnr = 0
test_ssim = 0
interpolate_psnr = 0
image_count = 0
upscale = 3

test_path = r"D:\PC\cyvision\SR_datasets\PlantSR_dataset\test"

for filename in os.listdir(test_path):
    if filename.endswith((".png",".jpg")):
        image_count+=1
        file_path = os.path.join(test_path, filename)
        
        hr_img = cv2.imread(file_path, cv2.IMREAD_COLOR).astype(np.float32) 
        h, w, _ = hr_img.shape

        ## make sure same dimension
        h -= h % upscale
        w -= w % upscale
        hr_img = hr_img[:h, :w]
        
        lr_image = cv2.resize(hr_img, (w // upscale, h // upscale), interpolation=cv2.INTER_LINEAR) 
        output = cv2.resize(lr_image, (w, h), interpolation=cv2.INTER_CUBIC) 
        
        psnr = calc_psnr(hr_img, output)
        ssim = calc_ssim(hr_img, output)
        test_psnr += psnr
        test_ssim += ssim

test_psnr = test_psnr/image_count
test_ssim = test_ssim/image_count

In [7]:
print('test psnr: {:.2f}'.format(test_psnr))
print('test ssim: {:.4f}'.format(test_ssim))

test psnr: 33.56
test ssim: 0.9128


In [67]:
## Test EDSR

In [15]:
from models.EDSR import EDSR
import torch
device = 'cuda'
model_path = r'outputs/edsr_x4_best.pth'
upscale = 4
model = EDSR(
    num_in_ch=3, num_out_ch=3,num_feat=64,num_block=16,upscale=upscale,res_scale=1)
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)

In [17]:
import cv2 as cv2
import numpy as np
import torch.nn.functional as F
import os

test_psnr = 0
test_ssim = 0
interpolate_psnr = 0
image_count = 0

test_path = "./data/PlantSR_dataset/test"

for filename in os.listdir(test_path):
    if filename.endswith((".png",".jpg")):
        print(image_count)
        image_count+=1
        file_path = os.path.join(test_path, filename)
        
        hr_img = cv2.imread(file_path, cv2.IMREAD_COLOR).astype(np.float32) 
        h, w, _ = hr_img.shape

        ## make sure same dimension
        h -= h % upscale
        w -= w % upscale
        hr_img = hr_img[:h, :w]
        
        lr_image = cv2.resize(hr_img, (w // upscale, h // upscale), interpolation=cv2.INTER_LINEAR) 
        lr_image = lr_image/255.
        lr_image = torch.from_numpy(np.transpose(lr_image[:, :, [2, 1, 0]],
                                                (2, 0, 1))).float()
        lr_image = lr_image.unsqueeze(0).to(device)
        
        
        with torch.no_grad():
            output = model(lr_image)

        output = output.data.squeeze().float().cpu().clamp_(0, 1).numpy()
        output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
        output = (output * 255.0)

        # if (output.shape[2] != hr_img.shape[2]) or (output.shape[3] != hr_img.shape[3]):
        #     output = F.interpolate(output, size=(h, w), mode='bilinear', align_corners=False)
            
        psnr = calc_psnr(hr_img, output)
        ssim = calc_ssim(hr_img, output)
        test_psnr += psnr
        test_ssim += ssim

test_psnr = test_psnr/image_count
test_ssim = test_ssim/image_count

In [20]:
print('test psnr: {:.2f}'.format(test_psnr))
print('test ssim: {:.4f}'.format(test_ssim))

test psnr: 37.36
test ssim: 0.9355


In [51]:
## test SwinIR

In [5]:
from models.network_swinir import SwinIR as net
import torch
device = 'cuda'
model_path = r'outputs/SwinIR_x3_30000_7.pth'
upscale = 3
model = net(upscale=upscale, in_chans=3, img_size=64, window_size=8,
            img_range=1., depths=[6, 6, 6, 6, 6, 6], embed_dim=180, num_heads=[6, 6, 6, 6, 6, 6],
            mlp_ratio=2, upsampler='pixelshuffle', resi_connection='1conv').to(device)
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)

In [2]:
from calulate_psnr_ssim import *

In [6]:
import cv2 as cv2
import numpy as np
import torch.nn.functional as F
import os

test_psnr = 0
test_ssim = 0
interpolate_psnr = 0
image_count = 0

test_path = "./data/PlantSR_dataset/test"

for filename in os.listdir(test_path):
    if filename.endswith((".png",".jpg")):
        image_count+=1
        print(image_count)
        file_path = os.path.join(test_path, filename)
        
        hr_img = cv2.imread(file_path, cv2.IMREAD_COLOR).astype(np.float32) 
        h, w, _ = hr_img.shape

        ## make sure same dimension
        h -= h % upscale
        w -= w % upscale
        hr_img = hr_img[:h, :w]
        
        lr_image = cv2.resize(hr_img, (w // upscale, h // upscale), interpolation=cv2.INTER_LINEAR) 
        lr_image = lr_image/255.
        lr_image = torch.from_numpy(np.transpose(lr_image[:, :, [2, 1, 0]],
                                                (2, 0, 1))).float()
        lr_image = lr_image.unsqueeze(0).to(device)
        
        with torch.no_grad():
            output = model(lr_image)

        output = output.data.squeeze().float().cpu().clamp_(0, 1).numpy()
        output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
        output = (output * 255.0)

        # if (output.shape[2] != hr_img.shape[2]) or (output.shape[3] != hr_img.shape[3]):
        #     output = F.interpolate(output, size=(h, w), mode='bilinear', align_corners=False)
            
        psnr = calc_psnr(hr_img, output)
        ssim = calc_ssim(hr_img, output)
        test_psnr += psnr
        test_ssim += ssim

test_psnr = test_psnr/image_count
test_ssim = test_ssim/image_count

In [4]:
print('test psnr: {:.2f}'.format(test_psnr))
print('test ssim: {:.4f}'.format(test_ssim))

test psnr: 34.65
test ssim: 0.9291


In [98]:
## test SRCNN

In [1]:
from models.SRCNN import SRCNN

In [2]:
from torch.utils.data import DataLoader
import torch.nn as nn
import torch

##  upscale = 2/3/4
upscale= 3
model_path = 'outputs/SRCNN_x3_5.pth'
device = 'cuda:0'

model = SRCNN()
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)

In [4]:
import cv2 as cv2
import numpy as np
import torch.nn.functional as F
import os

test_psnr = 0
test_ssim = 0
interpolate_psnr = 0
image_count = 0

test_path = "./data/PlantSR_dataset/test"

for filename in os.listdir(test_path):
    if filename.endswith((".png",".jpg")):
        image_count+=1
        file_path = os.path.join(test_path, filename)
        
        hr_img = cv2.imread(file_path, cv2.IMREAD_COLOR).astype(np.float32) 
        h, w, _ = hr_img.shape

        ## make sure same dimension
        h -= h % upscale
        w -= w % upscale
        hr_img = hr_img[:h, :w]
        
        lr_image = cv2.resize(hr_img, (w // upscale, h // upscale), interpolation=cv2.INTER_LINEAR) 
        lr_image = cv2.resize(lr_image, (w, h), interpolation=cv2.INTER_LINEAR)
        lr_image = lr_image/255.
        lr_image = torch.from_numpy(np.transpose(lr_image[:, :, [2, 1, 0]],
                                                (2, 0, 1))).float()
        lr_image = lr_image.unsqueeze(0).to(device)
        
        
        with torch.no_grad():
            output = model(lr_image)

        output = output.data.squeeze().float().cpu().clamp_(0, 1).numpy()
        output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
        output = (output * 255.0)

        # if (output.shape[2] != hr_img.shape[2]) or (output.shape[3] != hr_img.shape[3]):
        #     output = F.interpolate(output, size=(h, w), mode='bilinear', align_corners=False)
            
        psnr = calc_psnr(hr_img, output)
        ssim = calc_ssim(hr_img, output)
        test_psnr += psnr
        test_ssim += ssim

test_psnr = test_psnr/image_count
test_ssim = test_ssim/image_count

In [5]:
print('test psnr: {:.2f}'.format(test_psnr))
print('test ssim: {:.4f}'.format(test_ssim))

test psnr: 34.24
test ssim: 0.9190


In [1]:
## test RCAN

In [5]:
from models.RCAN import RCAN
import torch

upscale = 3
device = 'cuda'
model_path = r'outputs/RCAN_x3_6.pth'

class Args():
  def __init__(self,num_channel,num_feat,scale,n_resgroups,n_resblocks,reduction) :
    self.num_channel = num_channel
    self.num_feat = num_feat
    self.scale = scale
    self.n_resgroups = n_resgroups
    self.n_resblocks = n_resblocks
    self.reduction = reduction
args = Args(3,64,upscale,10,20,16)
model = RCAN(args)
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)

In [7]:
import cv2 as cv2
import numpy as np
import torch.nn.functional as F
import os

test_psnr = 0
test_ssim = 0
image_count = 0

test_path = "./data/PlantSR_dataset/test"

for filename in os.listdir(test_path):
    if filename.endswith((".png",".jpg")):
        image_count+=1
        file_path = os.path.join(test_path, filename)
        
        hr_img = cv2.imread(file_path, cv2.IMREAD_COLOR).astype(np.float32) 
        h, w, _ = hr_img.shape

        ## make sure same dimension
        h -= h % upscale
        w -= w % upscale
        hr_img = hr_img[:h, :w]
        
        lr_image = cv2.resize(hr_img, (w // upscale, h // upscale), interpolation=cv2.INTER_LINEAR) 
        lr_image = lr_image/255.
        lr_image = torch.from_numpy(np.transpose(lr_image[:, :, [2, 1, 0]],
                                                (2, 0, 1))).float()
        lr_image = lr_image.unsqueeze(0).to(device)
        
        
        with torch.no_grad():
            output = model(lr_image)

        output = output.data.squeeze().float().cpu().clamp_(0, 1).numpy()
        output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
        output = (output * 255.0)

        # if (output.shape[2] != hr_img.shape[2]) or (output.shape[3] != hr_img.shape[3]):
        #     output = F.interpolate(output, size=(h, w), mode='bilinear', align_corners=False)
            
        psnr = calc_psnr(hr_img, output)
        ssim = calc_ssim(hr_img, output)
        test_psnr += psnr
        test_ssim += ssim

test_psnr = test_psnr/image_count
test_ssim = test_ssim/image_count

In [8]:
print('test psnr: {:.2f}'.format(test_psnr))
print('test ssim: {:.4f}'.format(test_ssim))

test psnr: 35.18
test ssim: 0.9288


In [6]:
## test VDSR

In [1]:
from models.VDSR import VDSR

In [2]:
from torch.utils.data import DataLoader
import torch.nn as nn
import torch

##  upscale = 2/3/4
upscale= 2
model_path = 'outputs/VDSR_x2_2.pth'
device = 'cuda:0'

model = VDSR()
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)

In [4]:
import cv2 as cv2
import numpy as np
import torch.nn.functional as F
import os

test_psnr = 0
test_ssim = 0
image_count = 0

test_path = "./data/PlantSR_dataset/test"

for filename in os.listdir(test_path):
    if filename.endswith((".png",".jpg")):
        image_count+=1
        file_path = os.path.join(test_path, filename)
        
        hr_img = cv2.imread(file_path, cv2.IMREAD_COLOR).astype(np.float32) 
        h, w, _ = hr_img.shape

        ## make sure same dimension
        h -= h % upscale
        w -= w % upscale
        hr_img = hr_img[:h, :w]
        
        lr_image = cv2.resize(hr_img, (w // upscale, h // upscale), interpolation=cv2.INTER_LINEAR) 
        lr_image = cv2.resize(lr_image, (w, h), interpolation=cv2.INTER_LINEAR)
        
        lr_image = lr_image/255.
        lr_image = torch.from_numpy(np.transpose(lr_image[:, :, [2, 1, 0]],
                                                (2, 0, 1))).float()
        lr_image = lr_image.unsqueeze(0).to(device)
        
        
        with torch.no_grad():
            output = model(lr_image)

        output = output.data.squeeze().float().cpu().clamp_(0, 1).numpy()
        output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
        output = (output * 255.0)

        # if (output.shape[2] != hr_img.shape[2]) or (output.shape[3] != hr_img.shape[3]):
        #     output = F.interpolate(output, size=(h, w), mode='bilinear', align_corners=False)
            
        psnr = calc_psnr(hr_img, output)
        ssim = calc_ssim(hr_img, output)
        test_psnr += psnr
        test_ssim += ssim

test_psnr = test_psnr/image_count
test_ssim = test_ssim/image_count

In [5]:
print('test psnr: {:.2f}'.format(test_psnr))
print('test ssim: {:.4f}'.format(test_ssim))

test psnr: 40.31
test ssim: 0.9716


In [1]:
## test PlantSR

In [26]:
from models.PlantSR import PlantSR
# from PLtest import PlantSR
import torch

upscale = 2
device = 'cuda'
# model_path = r'outputs/PlantSR_x4_20000_10.pth'
model_path = r'outputs/PlantSR_x2_20000_1.pth'
device = torch.device('cuda:0')
if upscale == 4:
    model = PlantSR(scale=upscale,num_features=96,n_resgroups=16,n_resblocks=4,reduction=16)
if upscale == 2:
    model = PlantSR(scale=upscale,num_features=32,n_resgroups=16,n_resblocks=4,reduction=16)
if upscale == 3:
    model = PlantSR(scale=upscale,num_features=64,n_resgroups=16,n_resblocks=4,reduction=16)

model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)

In [7]:
import cv2 as cv2
import numpy as np
import torch.nn.functional as F
from calulate_psnr_ssim import *
import os

test_psnr = 0
test_ssim = 0
image_count = 0

test_path = "./data/PlantSR_dataset/test"

for filename in os.listdir(test_path):
    if filename.endswith((".png",".jpg")):
        image_count+=1
        print(image_count)
        file_path = os.path.join(test_path, filename)
        
        hr_img = cv2.imread(file_path, cv2.IMREAD_COLOR).astype(np.float32) 
        h, w, _ = hr_img.shape

        ## make sure same dimension
        h -= h % upscale
        w -= w % upscale
        hr_img = hr_img[:h, :w]
        
        lr_image = cv2.resize(hr_img, (w // upscale, h // upscale), interpolation=cv2.INTER_LINEAR) 
        lr_image = lr_image/255.
        lr_image = torch.from_numpy(np.transpose(lr_image[:, :, [2, 1, 0]],
                                                (2, 0, 1))).float()
        lr_image = lr_image.unsqueeze(0).to(device)
        
        
        with torch.no_grad():
            output = model(lr_image)

        output = output.data.squeeze().float().cpu().clamp_(0, 1).numpy()
        output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
        output = (output * 255.0)

        # if (output.shape[2] != hr_img.shape[2]) or (output.shape[3] != hr_img.shape[3]):
        #     output = F.interpolate(output, size=(h, w), mode='bilinear', align_corners=False)
            
        psnr = calc_psnr(hr_img, output)
        ssim = calc_ssim(hr_img, output)
        test_psnr += psnr
        test_ssim += ssim

test_psnr = test_psnr/image_count
test_ssim = test_ssim/image_count

In [29]:
print('test psnr: {:.2f}'.format(test_psnr))
print('test ssim: {:.4f}'.format(test_ssim))

test psnr: 40.33
test ssim: 0.9715
